In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Installing the necessary packages
!pip install dfply
!pip install ggplot
!pip install pandas==0.19.2

     |████████████████████████████████| 614kB 2.8MB/s 
     |████████████████████████████████| 2.2MB 2.8MB/s 
     |████████████████████████████████| 18.9MB 2.8MB/s 
ERROR: plotnine 0.5.1 has requirement pandas>=0.23.4, but you'll have pandas 0.19.2 which is incompatible.
ERROR: mizani 0.5.4 has requirement pandas>=0.23.4, but you'll have pandas 0.19.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=0.24.0, but you'll have pandas 0.19.2 which is incompatible.
ERROR: featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.19.2 which is incompatible.
ERROR: fbprophet 0.5 has requirement pandas>=0.23.4, but you'll have pandas 0.19.2 which is incompatible.
  Found existing installation: pandas 0.24.2
    Uninstalling pandas-0.24.2:
      Successfully uninstalled pandas-0.24.2


In [0]:
#Loading data from the google drive to colab notebook

# Please change the filename as per the location where the file is stored

filename = '/content/drive/My Drive/Packt_Colab/bank-full.csv'

In [0]:
# Importing necessary packages
from dfply import *
from ggplot import *

In [0]:
bankData = pd.read_csv(filename,sep=";")

In [4]:
# Relationship between housing and propensity for term deposits
(bankData >> group_by(X.housing,X.y) >> summarise(houseTot = X.y.count()))

,y,housing,houseTot
0,no,no,16727
1,yes,no,3354
2,no,yes,23195
3,yes,yes,1935


In [5]:
# Relationship between having a loan and propensity for term deposits
(bankData) >> group_by(X.loan,X.y) >> summarise(loanTot = X.y.count())

,y,loan,loanTot
0,no,no,33162
1,yes,no,4805
2,no,yes,6760
3,yes,yes,484


In [6]:
# Taking the quantiles for 25%, 50% and 75% of the balance data
import numpy as np
np.quantile(bankData['balance'],[0.25,0.5,0.75])

array([  72.,  448., 1428.])

In [7]:
# Creating new features for bank data based on the quantile values

bankData['balanceClass'] = 'Quant1'

bankData.balanceClass[(bankData['balance'] > 72) & (bankData['balance'] < 448)] = 'Quant2'

bankData.balanceClass[(bankData['balance'] > 448) & (bankData['balance'] < 1428)] = 'Quant3'

bankData.balanceClass[bankData['balance'] > 1428] = 'Quant4'

bankData.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,balanceClass
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,Quant4
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,Quant1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,Quant1
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,Quant4
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,Quant1


In [8]:
# Calculating the customers under each quantile 
balanceTot = (bankData >> group_by(X.balanceClass) >> summarise(balanceTot = X.y.count()))
balanceTot

,balanceClass,balanceTot
0,Quant1,11340
1,Quant2,11275
2,Quant3,11299
3,Quant4,11297


In [9]:
# Calculating the total customers categorised as per quantile and propensity classification
balanceProp = (bankData >> group_by(X.balanceClass,X.y) >> summarise(balanceCat = X.y.count()))
balanceProp

,y,balanceClass,balanceCat
0,no,Quant1,10517
1,yes,Quant1,823
2,no,Quant2,10049
3,yes,Quant2,1226
4,no,Quant3,9884
5,yes,Quant3,1415
6,no,Quant4,9472
7,yes,Quant4,1825


In [10]:
# Merging both the data frames
balanceComb = (pd.merge(balanceProp,balanceTot,left_on = ['balanceClass'],right_on = ['balanceClass']) >> mutate(catProp = (X.balanceCat/X.balanceTot)*100))
balanceComb

,y,balanceClass,balanceCat,balanceTot,catProp
0,no,Quant1,10517,11340,92.742504
1,yes,Quant1,823,11340,7.257496
2,no,Quant2,10049,11275,89.126386
3,yes,Quant2,1226,11275,10.873614
4,no,Quant3,9884,11299,87.476768
5,yes,Quant3,1415,11299,12.523232
6,no,Quant4,9472,11297,83.845269
7,yes,Quant4,1825,11297,16.154731
